# Linear Regression Consulting Project

Congratulations! You've been contracted by Hyundai Heavy Industries to help them build a predictive model for some ships. [Hyundai Heavy Industries](http://www.hyundai.eu/en) is one of the world's largest ship manufacturing companies and builds cruise liners.

You've been flown to their headquarters in Ulsan, South Korea to help them give accurate estimates of how many crew members a ship will require.

They are currently building new ships for some customers and want you to create a model and use it to predict how many crew members the ships will need.

Here is what the data looks like so far:

    Description: Measurements of ship size, capacity, crew, and age for 158 cruise
    ships.


    Variables/Columns
    Ship Name     1-20
    Cruise Line   21-40
    Age (as of 2013)   46-48
    Tonnage (1000s of tons)   50-56
    passengers (100s)   58-64
    Length (100s of feet)  66-72
    Cabins  (100s)   74-80
    Passenger Density   82-88
    Crew  (100s)   90-96
    
It is saved in a csv file for you called "cruise_ship_info.csv". Your job is to create a regression model that will help predict how many crew members will be needed for future ships. The client also mentioned that they have found that particular cruise lines will differ in acceptable crew counts, so it is most likely an important feature to include in your analysis! 

Once you've created the model and tested it for a quick check on how well you can expect it to perform, make sure you take a look at why it performs so well!

In [1]:
import findspark
findspark.init('/home/richard/spark-3.2.0-bin-hadoop3.2')

In [2]:
from pyspark.sql import SparkSession

In [22]:
spark = SparkSession.builder.appName('project').getOrCreate()

In [4]:
from pyspark.ml.regression import LinearRegression

In [5]:
data = spark.read.csv('cruise_ship_info.csv',inferSchema=True,header=True)

In [6]:
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler, StringIndexer

In [7]:
data.columns

['Ship_name',
 'Cruise_line',
 'Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density',
 'crew']

In [8]:
data.printSchema()

root
 |-- Ship_name: string (nullable = true)
 |-- Cruise_line: string (nullable = true)
 |-- Age: integer (nullable = true)
 |-- Tonnage: double (nullable = true)
 |-- passengers: double (nullable = true)
 |-- length: double (nullable = true)
 |-- cabins: double (nullable = true)
 |-- passenger_density: double (nullable = true)
 |-- crew: double (nullable = true)



In [9]:
indexer = StringIndexer(inputCol="Cruise_line", outputCol="cruise_cat")
indexed = indexer.fit(data).transform(data)

In [31]:
indexed.show()

+-----------+-----------+---+------------------+----------+------+------+-----------------+----+----------+
|  Ship_name|Cruise_line|Age|           Tonnage|passengers|length|cabins|passenger_density|crew|cruise_cat|
+-----------+-----------+---+------------------+----------+------+------+-----------------+----+----------+
|    Journey|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|      16.0|
|      Quest|    Azamara|  6|30.276999999999997|      6.94|  5.94|  3.55|            42.64|3.55|      16.0|
|Celebration|   Carnival| 26|            47.262|     14.86|  7.22|  7.43|             31.8| 6.7|       1.0|
|   Conquest|   Carnival| 11|             110.0|     29.74|  9.53| 14.88|            36.99|19.1|       1.0|
|    Destiny|   Carnival| 17|           101.353|     26.42|  8.92| 13.21|            38.36|10.0|       1.0|
|    Ecstasy|   Carnival| 22|            70.367|     20.52|  8.55|  10.2|            34.29| 9.2|       1.0|
|    Elation|   Carnival| 15

In [10]:
assembler = VectorAssembler(inputCols=['Age',
 'Tonnage',
 'passengers',
 'length',
 'cabins',
 'passenger_density', 'cruise_cat'],outputCol='features')

In [11]:
output = assembler.transform(indexed)

In [12]:
final_data = output.select('features','crew')

In [13]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [14]:
lr = LinearRegression(labelCol='crew')

In [23]:
lrModel = lr.fit(train_data)

21/12/19 16:05:41 WARN Instrumentation: [f95f1c67] regParam is zero, which might cause numerical instability and overfitting.


In [16]:
test_results = lrModel.evaluate(test_data)

In [27]:
test_results.rootMeanSquaredError

1.476174271795342

In [25]:
test_results.r2

0.8722458447455839

In [26]:
test_results.r2adj

0.8433981322687802

In [29]:
test_results.residuals.show()

+--------------------+
|           residuals|
+--------------------+
| 0.30257948569481385|
|  0.0725629005655879|
| -0.7751049627497322|
|  0.7035593327451384|
| -0.2817707563013041|
|  0.8849518469307949|
| -0.4327869561436817|
|  1.1657038289894537|
|-0.06707430866646646|
|  1.4692847055535752|
|-0.11643090334180428|
| -0.2758099352034886|
| -0.3186452437131049|
|   7.216509848091043|
| -1.0229248767328727|
|-0.03926873765955019|
| -1.0150145019870198|
|  -0.412669174738884|
|-0.16373086751441335|
| 0.36021134919127906|
+--------------------+
only showing top 20 rows



In [30]:
test_data.describe().show()

+-------+-----------------+
|summary|             crew|
+-------+-----------------+
|  count|               39|
|   mean|8.772307692307692|
| stddev|4.183989824342603|
|    min|              1.6|
|    max|             21.0|
+-------+-----------------+

